In [ ]:
# Install cryptography if not already available
!pip install cryptography

# Import libraries
import os
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes, padding
from base64 import urlsafe_b64encode, urlsafe_b64decode
import secrets

# Constants
KEY_LENGTH = 32  # AES-256
IV_LENGTH = 16   # AES block size
SALT_LENGTH = 16
ITERATIONS = 100_000

def derive_key(password: bytes, salt: bytes) -> bytes:
    kdf = PBKDF2HMAC(
        algorithm=hashes.SHA256(),
        length=KEY_LENGTH,
        salt=salt,
        iterations=ITERATIONS,
        backend=default_backend()
    )
    return kdf.derive(password)

def encrypt_file(input_path, output_path, password):
    password = password.encode()
    salt = secrets.token_bytes(SALT_LENGTH)
    key = derive_key(password, salt)
    iv = secrets.token_bytes(IV_LENGTH)

    cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())
    encryptor = cipher.encryptor()
    padder = padding.PKCS7(128).padder()

    with open(input_path, 'rb') as f:
        data = f.read()
    padded_data = padder.update(data) + padder.finalize()
    encrypted = encryptor.update(padded_data) + encryptor.finalize()

    with open(output_path, 'wb') as f:
        f.write(salt + iv + encrypted)

    print(f"🔐 Encrypted: {output_path}")

def decrypt_file(input_path, output_path, password):
    password = password.encode()

    with open(input_path, 'rb') as f:
        file_data = f.read()

    salt = file_data[:SALT_LENGTH]
    iv = file_data[SALT_LENGTH:SALT_LENGTH+IV_LENGTH]
    encrypted_data = file_data[SALT_LENGTH+IV_LENGTH:]

    key = derive_key(password, salt)
    cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())
    decryptor = cipher.decryptor()
    unpadder = padding.PKCS7(128).unpadder()

    decrypted_padded = decryptor.update(encrypted_data) + decryptor.finalize()
    decrypted = unpadder.update(decrypted_padded) + unpadder.finalize()

    with open(output_path, 'wb') as f:
        f.write(decrypted)

    print(f"🔓 Decrypted: {output_path}")

In [ ]:
encrypt_file("mysecret.txt", "mysecret.encrypted", "MyStrongPassword123!")


🔐 Encrypted: mysecret.encrypted


In [ ]:
decrypt_file("mysecret.encrypted", "mysecret_decrypted.txt", "MyStrongPassword123!")


🔓 Decrypted: mysecret_decrypted.txt


In [ ]:
# Read the decrypted file to verify
with open("mysecret_decrypted.txt", "r") as f:
    print("📝 Decrypted content:", f.read())


📝 Decrypted content: This is a top secret message.


In [ ]:
!pip install requests beautifulsoup4


In [ ]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

class VulnScanner:
    def __init__(self, url):
        self.url = url
        self.session = requests.Session()

    def check_headers(self):
        print("🛡️ Checking security headers...")
        resp = self.session.get(self.url)
        missing = []
        headers = {
            "X-Frame-Options": "Prevents clickjacking",
            "X-XSS-Protection": "Enables XSS filter",
            "Content-Security-Policy": "Helps prevent XSS",
            "Strict-Transport-Security": "Enforces HTTPS",
        }
        for header in headers:
            if header not in resp.headers:
                missing.append((header, headers[header]))
        if missing:
            print("[!] Missing security headers:")
            for h, desc in missing:
                print(f"  - {h}: {desc}")
        else:
            print("[+] All common security headers present.")

    def check_csrf_in_forms(self):
        print("\n🧾 Checking for CSRF tokens in forms...")
        try:
            resp = self.session.get(self.url)
            soup = BeautifulSoup(resp.text, "html.parser")
            forms = soup.find_all("form")
            if not forms:
                print("[*] No forms found.")
                return
            for i, form in enumerate(forms):
                inputs = form.find_all("input")
                has_token = any("csrf" in (inp.get("name") or "").lower() for inp in inputs)
                action = form.get("action")
                full_action = urljoin(self.url, action) if action else self.url
                if has_token:
                    print(f"[+] Form {i+1} ({full_action}) includes CSRF token.")
                else:
                    print(f"[!] Form {i+1} ({full_action}) is missing CSRF token.")
        except Exception as e:
            print("Error checking CSRF tokens:", e)

    def test_reflected_xss(self):
        print("\n🔍 Testing for reflected XSS...")
        test_payload = "<script>alert(1)</script>"
        try:
            test_url = self.url + "?q=" + test_payload
            resp = self.session.get(test_url)
            if test_payload in resp.text:
                print(f"[!] Possible reflected XSS at: {test_url}")
            else:
                print("[+] No reflected XSS detected.")
        except Exception as e:
            print("Error testing for XSS:", e)

    def run_all(self):
        print(f"\n🔎 Scanning: {self.url}")
        self.check_headers()
        self.check_csrf_in_forms()
        self.test_reflected_xss()


In [ ]:
# Replace with your test site (must be one you own or have permission to scan!)
scanner = VulnScanner("http://example.com")
scanner.run_all()



🔎 Scanning: http://example.com
🛡️ Checking security headers...
[!] Missing security headers:
  - X-Frame-Options: Prevents clickjacking
  - X-XSS-Protection: Enables XSS filter
  - Content-Security-Policy: Helps prevent XSS
  - Strict-Transport-Security: Enforces HTTPS

🧾 Checking for CSRF tokens in forms...
[*] No forms found.

🔍 Testing for reflected XSS...
[+] No reflected XSS detected.
